In [1]:
import pandas as pd
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

service=Service('C:/Users/Manych/chromium/chromedriver.exe')

In [8]:
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuel_type', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id', 'vehicleTransmission',
       'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'offerprice']


In [15]:
# ticket_url ='https://auto.ru/cars/used/sale/audi/q3/1114681238-eeb9b220/'
# ticket_url ='https://auto.ru/cars/used/sale/bmw/5er/1106553723-4fdacc7a/'
# ticket_url = 'https://auto.ru/cars/used/sale/bmw/5er/1105820581-c8d41ca3/'
ticket_url = 'https://auto.ru/cars/used/sale/skoda/fabia/1114927617-05b1728c/'

In [16]:
if 1:
    # получем html карточки текщего обьявления
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    driver.get(ticket_url)
    # Находим и кликаем 'Все опции'
    butt = driver.find_element(By.CLASS_NAME, 'ComplectationGroupsDesktop__cut') 
    butt.click()    
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  
    if 1:
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            t =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
            
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            tt = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN     
    
        try: 
            ttt = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN 
            
t,tt,ttt

('1.4', '75', 'Бензин')

In [10]:
# if 1:
def get_features_from_ticket(ticket_url, service):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service : обьект selenium.webdriver.chrome.service
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    driver.get(ticket_url)
    # Находим и кликаем 'Все опции'
    butt = driver.find_element(By.CLASS_NAME, 'ComplectationGroupsDesktop__cut') 
    butt.click()    
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  

    # проверка корректности результата BeautifulSoup
    if ticket_bs:
    # получение признаков из карточки текущего обьявления
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN

        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[2].text.strip().upper()
        except Exception: features['brand'] = np.NaN
            
        # car_url
        features['car_url'] = ticket_url
            
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN 
            
        # description
        try:
            rows = ticket_bs.find('div',class_='CardDescriptionHTML').find_all('span')
            features['description'] = '\n'.join([row.text for row in rows])
        except Exception: features['description'] = np.NaN
        
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
            
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN            

        # equipment_dict ------ pd.Series(ttt.values()).sum()
        try: 
            complectation_groups = ticket_bs.find(
                'div',class_='ComplectationGroupsDesktop__row').find_all(
                'div',class_='ComplectationGroupsDesktop__group')
            features['equipment_dict'] = {
                group.text.split('•')[0]: group.text.split('•')[1:] for group in complectation_groups}
        except Exception: features['equipment_dict'] = np.NaN             
  
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
            
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN            
            
        # model_name
        try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[3].text.strip()
        except Exception: features['model_name'] = np.NaN            
            
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN

        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
            
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
            
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN            
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'
            
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN            
            
        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN               
            
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN            
            
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN         
            
        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN

        # modelDate              
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[2].text.strip() + ' ' +
                                     modelDate_tag[3].text.strip() + ' ' +
                                     modelDate_tag[4].text.strip())  
        except Exception: features['modelDate'] = np.NaN   
    return features

In [11]:
get_features_from_ticket(ticket_url,service)

<ipython-input-10-1c5e876aa992>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  features = pd.Series(index = data_columns)


bodyType                                                 универсал 5 дв.
brand                                                                BMW
car_url                https://auto.ru/cars/used/sale/bmw/5er/1106553...
color                                                              синий
engineDisplacement                                                   2.5
enginePower                                                          143
equipment_dict         {'Безопасность': ['Антиблокировочная система (...
fuel_type                                                            NaN
mileage                                                           341000
modelDate                                          BMW 5 серии III (E34)
model_name                                                       5 серии
numberOfDoors                                                          5
productionDate                                                      1992
sell_id                                            

# ============================================================


In [41]:
# get_features_from_ticket(ticket_url,service)

In [13]:
ticket_url = 'https://auto.ru/cars/used/sale/audi/e_tron/1106486096-182f9d32/'

In [14]:
get_features_from_ticket(ticket_url)

<ipython-input-7-0bf35711b29d>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  features = pd.Series(index = data_columns)


bodytype                                        NaN
brand                           ПРОДАЖА АВТОМОБИЛЕЙ
car_url                                         NaN
color                                         синий
engineDisplacement                              NaN
enginePower                                     NaN
equipment_dict                                   {}
fuel_type                                       NaN
mileage                                        4000
modelDate              Продажа автомобилей Б/у Audi
model_name                                      Б/у
numberOfDoors                                     5
productionDate                                 2020
sell_id                                  1106486096
vehicleTransmission                  автоматическая
vendor                                          NAN
Владельцы                                1 владелец
ПТС                                        Оригинал
Привод                                       полный
Руль        

# ============================================================

In [17]:
%%time
driver = webdriver.Chrome(service=s)
driver.maximize_window()
url = 'https://auto.ru/cars/used/sale/bmw/5er/1105850176-392bf056/'
driver.get(url)
# Получаем содержимое html-страницы:
page = driver.execute_script("return document.body.innerHTML;")
# Закрываем процесс браузера:
driver.quit()

soup = BeautifulSoup(page, 'html.parser')
[child.get_text(': ').replace('\xa0', ' ') for child in soup.find('ul', class_='CardInfo').children]

Wall time: 10.1 s


['год выпуска: 2017',
 'Пробег: 159 857 км',
 'Кузов: универсал 5 дв.',
 'Цвет: белый',
 'Двигатель: 2.0 л / 190 л.с. / : Дизель',
 'Комплектация: 48 опций',
 'Коробка: автоматическая',
 'Привод: задний',
 'Руль: Левый',
 'Состояние: Не требует ремонта',
 'Владельцы: 1 владелец',
 'ПТС: Оригинал',
 'Таможня: Растаможен',
 'Гарантия: До ноября 2023',
 'Обмен: Рассмотрю варианты',
 'VIN: WBA**************']

In [51]:
def get_features_from_ticket(ticket_url, service):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service : обьект selenium.webdriver.chrome.service
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    driver.get(ticket_url)
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  
    # проверка корректности резкльтата BeautifulSoup
    if ticket_bs:
    
    # получение признаков из карточки текущего обьявления
    #
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN
           
        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip().upper()
        except Exception: features['brand'] = np.NaN
    
#         car_url
#         features['car_url'] = ticket_url
    
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN
    
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
          
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN
    
        # equipment_dict
        equipment_dict = dict()
        try:
            complectation = ticket_bs.find('section',class_='CardComplectation')
            complectation_items = complectation.find_all('div',class_="ComplectationGroups__group")
            for item in complectation_items:
                item_name = item.find('span',class_="ComplectationGroups__itemName").text
                item_content_tag = item.find_all('li',class_="ComplectationGroups__itemContentEl")
                item_content = [x.text for x in item_content_tag]
                equipment_dict[item_name] = item_content 
            features['equipment_dict'] = equipment_dict
        except Exception: features['equipment_dict'] = np.NaN
    
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
        
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN
    
        # modelDate
        ## заполняется названием марки, модели и поколением модели - следует далее заменить на год модели/поколения
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[0].text.strip() + ' ' +
                                     modelDate_tag[1].text.strip() + ' ' +
                                     modelDate_tag[2].text.strip())  
        except Exception: features['modelDate'] = np.NaN
    
        # model_name
        try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[1].text.strip()
        except Exception: features['model_name'] = np.NaN
     
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN
    
        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
    
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
    
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'
        
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN

        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN
        
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN
    
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN
    
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN

        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN
            
    return features


In [52]:
get_features_from_ticket(ticket_url,service)

<ipython-input-51-4977f736e81e>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  features = pd.Series(index = data_columns)


bodytype                                        NaN
brand                           ПРОДАЖА АВТОМОБИЛЕЙ
car_url                                         NaN
color                                     оранжевый
engineDisplacement                              1.4
enginePower                                     150
equipment_dict                                  NaN
fuel_type                                       NaN
mileage                                       13000
modelDate              Продажа автомобилей Б/у Audi
model_name                                      Б/у
numberOfDoors                                     5
productionDate                                 2019
sell_id                                  1114614831
vehicleTransmission                роботизированная
vendor                                          NAN
Владельцы                                1 владелец
ПТС                                        Оригинал
Привод                                     передний
Руль        

def get_features_from_ticket(ticket_url, service):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service :
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    driver.get(ticket_url)
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  
    # проверка корректности резкльтата BeautifulSoup
    if ticket_bs:
    
    # получение признаков из карточки текущего обьявления
    #
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN
           
        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip().upper()
        except Exception: features['brand'] = np.NaN
    
#         car_url
#         features['car_url'] = ticket_url
    
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN
    
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
          
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN
    
        # equipment_dict
        equipment_dict = dict()
        try:
            complectation = ticket_bs.find('section',class_='CardComplectation')
            complectation_items = complectation.find_all('div',class_="ComplectationGroups__group")
            for item in complectation_items:
                item_name = item.find('span',class_="ComplectationGroups__itemName").text
                item_content_tag = item.find_all('li',class_="ComplectationGroups__itemContentEl")
                item_content = [x.text for x in item_content_tag]
                equipment_dict[item_name] = item_content 
            features['equipment_dict'] = equipment_dict
        except Exception: features['equipment_dict'] = np.NaN
    
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
        
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN
    
        # modelDate
        ## заполняется названием марки, модели и поколением модели - следует далее заменить на год модели/поколения
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[0].text.strip() + ' ' +
                                     modelDate_tag[1].text.strip() + ' ' +
                                     modelDate_tag[2].text.strip())  
        except Exception: features['modelDate'] = np.NaN
    
        # model_name
        try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[1].text.strip()
        except Exception: features['model_name'] = np.NaN
     
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN
    
        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
    
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
    
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'
        
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN

        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN
        
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN
    
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN

        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN
            
    return features
